# ClickStream - Getting Started - Explorations

This post gives an introduction to working with the [Wikipedia Clickstream](http://figshare.com/articles/Wikipedia_Clickstream/1305770) dataset. It shows how people get to a Wikipedia article and what articles they click on next. In other words, it gives a weighted network of articles, where each edge weight corresponds to how often people navigate from one page to another. To give an example, consider the figure below, which shows incoming and outgoing traffic to the "London" article.

In [36]:
from IPython.display import IFrame
IFrame('London_Sankey.html', width=950, height=450)

The example shows that most people found the "London" page through Google Search and that only a small fraction of readers went on to another article. Before diving into some examples of working with the data, let me give a more detailed explanation of how the data was collected.

### Data Preparation

The data contains counts of (referer, resource) pairs extracted from the request logs of English Wikipedia. When a client requests a resource by following a link or performing a search, the URI of the webpage that linked to the resource is included with the request in an HTTP header called the "referer". This data captures 22 million (referer, resource) pairs from a total of 3.2 billion requests collected during the month of February 2015.

The dataset only includes requests for articles in the [main namespace](https://en.wikipedia.org/wiki/Wikipedia:Namespace) of the desktop version of English Wikipedia.

Referers were [mapped](https://github.com/ewulczyn/wmf/blob/f76d43eee9e3e9756cd6a062a11e382fd4425edf/clickstream/oozie/hive_query.sql#L152-L203) to a fixed set of values corresponding to internal traffic or external traffic from one of the top 5 global traffic sources to English Wikipedia, based on this scheme:
    - an article in the main namespace of English Wikipedia -> the article title
    - any Wikipedia page that is not in the main namespace of English Wikipedia -> `other-wikipedia`
    - an empty referer -> `other-empty`
    - a page from any other Wikimedia project -> `other-internal`
    - Google -> `other-google`
    - Yahoo -> `other-yahoo`
    - Bing -> `other-bing`
    - Facebook -> `other-facebook`
    - Twitter -> `other-twitter`
    - anything else -> `other-other`

 
MediaWiki Redirects are used to forward clients from one page name to another. They can be useful if a particular article is referred to by multiple names, or has alternative punctuation, capitalization or spellings. Requests for pages that get redirected where mapped to the page they redirect to. For example, requests for 'Obama' redirect to the 'Barack_Obama' page. Redirects where resolved using a snapshot of the redirects table from March 1 2015.

Redlinks are are links to an article that does not exist. Either the article was deleted after the creation of the link or the author intended to signal the need for such an article. Requests for redlinks are included in the data. 

We attempt to exclude spider traffic by classifying user agents with the [ua-parser](https://github.com/tobie/ua-parser) library and a few additonal Wikipedia specific filters. Furthermore, we attempt to filter out traffic from bots that request a page and then request all or most of the links on that page (BFS traversal) by setting a threshold on the rate at which a client can requests articles with the same referer. Requests that where made at too high of a rate get discarded. For the exact details, see [here](https://github.com/ewulczyn/wmf/blob/f76d43eee9e3e9756cd6a062a11e382fd4425edf/clickstream/oozie/throttle.py). The threshold is quite high to avoid excluding human readers who open tabs as they read. As a result requests from slow moving bots are likely to remain in the data. More sophisticated bot detection, that evaluates the clients entire request graph is an avenue of future work.

Finally, any `(referer, resource)` pair with 10 or fewer observations was removed from the dataset. 


### Format
The data includes the following 6 fields:

- **prev_id:** if the referer does not correspond to an article in the main namespace of English Wikipedia, this value will be empty. Otherwise, it contains the unique MediaWiki page ID of the article corresponding to the referer i.e. the previous article the client was on
- **curr_id:** the MediaWiki unique page ID of the article the client requested 
- **n:** the number of occurrences of the `(referer, resource)` pair
- **prev_title:** the result of mapping the referer URL to the fixed set of values described above
- **curr_title:** the title of the article the client requested
- **type** 
- "link" if the referer and request are both articles and the referer links to the request
    - "redlink" if the referer is an article and links to the request, but the request is not in the produiction enwiki.page table
    - "other" if the referer and request are both articles but the referer does not link to the request. This can happen when clients search or spoof their refer

# Getting to know the Data

There are various quirks in the data due to the dynamic nature of the network of articles in English Wikipedia and the prevalence of requests from automata. The following section gives a brief overview of the data fields and caveats that need to be kept in mind.



### Loading the Data
First, let's load the data into a pandas DataFrame. 

You will need to download 2015_02_filtered_en_clickstream.tsv into the same directory as this Python file.

In [37]:
import pandas as pd
df = pd.read_csv("2015_02_filtered_en_clickstream.tsv", sep='\t', header=0)

In [38]:
#we won't use ids here, so lets discard them
df = df[['prev_title', 'curr_title', 'n', 'type']]
df.columns = ['prev', 'curr', 'n', 'type']

In [39]:
df.columns

Index(['prev', 'curr', 'n', 'type'], dtype='object')

### Top articles
It has been possible to estimate which pages get the most pageviews per month from the public pageview dumps that WMF releases. 

In [40]:
df.groupby('curr').sum().sort_values('n', ascending=False)[:15]

,n
curr,
87th_Academy_Awards,2559794
Fifty_Shades_of_Grey,2326175
Alive,2244781
Chris_Kyle,1709341
Fifty_Shades_of_Grey_(film),1683892
Deaths_in_2015,1614577
Birdman_(film),1545842
Islamic_State_of_Iraq_and_the_Levant,1406530
Stephen_Hawking,1384193


In [41]:
view25 = df.groupby('curr').sum().sort_values('n', ascending=False)[24:25].index.values[0]

view25

'Jamie_Dornan'

The most requested pages tend to be about media that was popular in April. 

### Top Referers
The clickstream data aslo lets us investigate who the top referers to Wikipedia are:

In [42]:
df.groupby('prev').sum().sort_values('n', ascending=False)[:15]

,n
prev,
other-google,16079189
other-empty,4258173
other-wikipedia,3391566
other-facebook,2314026
87th_Academy_Awards,1680675
Main_Page,1573391
Fifty_Shades_of_Grey_(film),1375687
Fifty_Shades_of_Grey,1146401
Deaths_in_2015,1135121


In [43]:
refers16 = df.groupby('prev').sum().sort_values('n', ascending=False)[15:16].index.values[0]

refers16

'Islamic_State_of_Iraq_and_the_Levant'

The top referer by a large margin is Google. Next comes refererless traffic (usually clients using HTTPS). Then come other language Wikipedias and pages in English Wikipedia that are not in the main (i.e. article) namespace. Bing directs significanlty more traffic to Wikipedia than Yahoo. Social media referals are tiny compared to Google, with twitter leading to 10x more requests to Wikipedia than Facebook. 

### Trending on Social Media
Lets look at what articles where trending on Twitter:

In [44]:
df_twitter = df[df['prev'] == 'other-twitter']
df_twitter.groupby('curr').sum().sort_values('n', ascending=False)[:5]

,n
curr,
Facebook,30624
Fifty_Shades_of_Grey,4950
Chinese_New_Year,4170
Fifty_Shades_of_Grey_(film),2650
Islamic_State_of_Iraq_and_the_Levant,1906


In [45]:
trend7 = df_twitter.groupby('curr').sum().sort_values('n', ascending=False)[5:6].index.values[0]

trend7

"Stayin'_Alive"

I'm have no explanations for this, but if you find any of the tweets linking to these article, I would be curious to see why they got so many click throughs. 

### Most Requested Missing Pages
Next lets look at the most popular redinks. Redlinks are links to a Wikipedia page that does not exist, either because it has been deleted, or because the author is anticipating the creation of the page. Seeing which redlinks are the most viewed is interesting because it gives some indication about demand for missing content. Since the set of pages and links is constantly changing, the labeling of redlinks is not an exact science. In this case, I used the page and links tables from Feb 28th to mark a page as a redlink.

In [46]:
df_redlinks = df[df['type'] == 'redlink']
df_redlinks.groupby('curr').sum().sort_values('n', ascending=False)[:5]

,n
curr,
Ben_Woolf,2764
June_Fairchild,1485
Tim_Ford_(politician),480
Bobby_Emmons,414
John_Crocker_(actor),379


In [47]:
df_redlinks[df_redlinks['prev'] == refers16]

,prev,curr,n,type
6309,Islamic_State_of_Iraq_and_the_Levant,BNPT,41,redlink
19114,Islamic_State_of_Iraq_and_the_Levant,Hafiz_Saeed_Khan,76,redlink
35732,Islamic_State_of_Iraq_and_the_Levant,Roméo_Dallaire_Child_Soldiers_Initiative,58,redlink
37090,Islamic_State_of_Iraq_and_the_Levant,Shabak_Militia,68,redlink
37539,Islamic_State_of_Iraq_and_the_Levant,Sinjar_Defence_Force,13,redlink
46135,Islamic_State_of_Iraq_and_the_Levant,Yekîneyên_Berxwedana_Şingal,13,redlink
50633,Islamic_State_of_Iraq_and_the_Levant,Abu_Salim_Martyrs_Brigade,20,redlink
87826,Islamic_State_of_Iraq_and_the_Levant,Sinjar_Resistance_Units,27,redlink


In [48]:
df[(df['prev'] == refers16) & (df['type'] == 'redlink')]

,prev,curr,n,type
6309,Islamic_State_of_Iraq_and_the_Levant,BNPT,41,redlink
19114,Islamic_State_of_Iraq_and_the_Levant,Hafiz_Saeed_Khan,76,redlink
35732,Islamic_State_of_Iraq_and_the_Levant,Roméo_Dallaire_Child_Soldiers_Initiative,58,redlink
37090,Islamic_State_of_Iraq_and_the_Levant,Shabak_Militia,68,redlink
37539,Islamic_State_of_Iraq_and_the_Levant,Sinjar_Defence_Force,13,redlink
46135,Islamic_State_of_Iraq_and_the_Levant,Yekîneyên_Berxwedana_Şingal,13,redlink
50633,Islamic_State_of_Iraq_and_the_Levant,Abu_Salim_Martyrs_Brigade,20,redlink
87826,Islamic_State_of_Iraq_and_the_Levant,Sinjar_Resistance_Units,27,redlink


### Searching Within Wikipedia

 Usually, clients navigate from one article to another through follwing a link. The other prominent case is search. The article from which the user searched is also passed as the referer to the found article. Hence, you will find a high count of `(Wikipedia, Chris_Kyle)` tuples. People went to the "Wikipedia" article to search for "Chris_Kyle". There is not a link to the "Chris_Kyle" article from the "Wikipedia" article. Finally, it is possible that the client messed with their referer header. The vast majority of requests with an internal referer correspond to a true link.

In [49]:
df_search = df[df['type'] == 'other']
df_search = df_search[df_search.prev.str.match("^other.*").apply(bool) == False]

df_search

print ("Number of searches/ incorrect referers: %d" % df_search.n.sum())

Number of searches/ incorrect referers: 1656796


In [50]:
df_link = df[df['type'] == 'link']
df_link =  df_link[df_link.prev.str.match("^other.*").apply(bool) == False]
print ("Number of links followed: %d" % df_link.n.sum())

Number of links followed: 19947182


### Inflow vs Outflow

You might be tempted to think that there can't be more traffic going out of a node than going into a node. This is not true for two reasons. People will follow links in multiple tabs as they read an article. Hence, a single pageview can lead to multiple records with that page as the referer. The data is also certain to include requests from bots which we did not correctly filter out. Bots will often follow most if not all the links in the article. Lets look at the ratio of incoming to outgoing links for the most requested pages.

In [51]:
df_in = df.groupby('curr').sum()  # pageviews per article
df_in.columns = ['in_count',]
df_out = df.groupby('prev').sum() # link clicks per article
df_out.columns = ['out_count',]
df_in_out = df_in.join(df_out)
df_in_out['ratio'] = df_in_out['out_count']/df_in_out['in_count'] #compute ratio if outflow/infow

In [52]:
df_in_out.sort_values('in_count', ascending = False)[:3]

,in_count,out_count,ratio
curr,,,
87th_Academy_Awards,2559794,1680675.0,0.656567
Fifty_Shades_of_Grey,2326175,1146401.0,0.492827
Alive,2244781,3480.0,0.001550


Looking at the pages with the highest ratio of outgoing to incoming traffic reveals how messy the data is, even after the carefull data preparation described above.

In [54]:
df_in_out.sort_values('ratio', ascending = False)[:3]

,in_count,out_count,ratio
curr,,,
Lists_of_films,13,8077.0,621.307692
Nimoy,21,4913.0,233.952381
List_of_awards_and_nominations_received_by_Julianne_Moore,15,3306.0,220.400000


In [55]:
df_in_out.sort_values('in_count', ascending = False)[8:9]

,in_count,out_count,ratio
curr,,,
Stephen_Hawking,1384193,644088.0,0.465317


In [56]:
df_in_out.sort_values('ratio', ascending = False)[7:8]

,in_count,out_count,ratio
curr,,,
The_Weeknd_discography,11,1333.0,121.181818


### Filtering Traversals

All of these pages have more traversals of a single link than they have requests for the page to begin with.  As a post processing step, we might enforce that there can't be more traversals of a link than there where requests to the page. Better bot filtering should help reduce this issue in the future.

In [57]:
df_post = pd.merge(df, df_in, how='left', left_on='prev', right_index=True)
df_post['n'] = df_post[['n', 'in_count']].min(axis=1)
del df_post['in_count']

In [58]:
df_post.head()

,prev,curr,n,type
0,List_of_Academy_Awards_ceremonies,Crocodile_Dundee,59.0,link
1,2015_in_film,Crocodile_Dundee,66.0,link
2,59th_Academy_Awards,Crocodile_Dundee,18.0,link
3,Deaths_in_2015,Crocodile_Dundee,1283.0,link
4,other-facebook,I_AM_Activity,33.0,other


In [61]:
df_in = df_post.groupby('curr').sum()  # pageviews per article
df_in
df_in.columns = ['in_count',]
df_out = df_post.groupby('prev').sum() # link clicks per article
df_out.columns = ['out_count',]
df_in_out = df_in.join(df_out)
df_in_out['ratio'] = df_in_out['out_count']/df_in_out['in_count']

In [62]:
df_in_out.sort_values('ratio', ascending = False)[:5]

,in_count,out_count,ratio
curr,,,
Wiki,36.0,790.0,21.944444
Wikipedia,318.0,4740.0,14.905660
List_of_awards_and_nominations_received_by_Meryl_Streep,80.0,797.0,9.962500
Main_Page,89834.0,793392.0,8.831756
February_2015,15.0,101.0,6.733333


### What is this code doing (#1)?

In [63]:
df_norm = pd.merge(df_post, df_out, how='left', left_on='prev', right_index=True)

In [65]:
df_norm = df_norm.sort_values(['prev', 'curr'])

In [67]:
df_norm['w'] = df_norm['n'] / df_norm['out_count']

In [69]:
df_norm.head(40)

,prev,curr,n,type,out_count,w
57171,%s,Birdman_(film),13.0,other,119.0,0.109244
8518,%s,Bruce_Jenner,13.0,other,119.0,0.109244
59721,%s,Chris_Kyle,18.0,other,119.0,0.151261
65189,%s,Fifty_Shades_of_Grey,20.0,other,119.0,0.168067
65915,%s,Fifty_Shades_of_Grey_(film),22.0,other,119.0,0.184874
25048,%s,Leonard_Nimoy,20.0,other,119.0,0.168067
89314,%s,Stephen_Hawking,13.0,other,119.0,0.109244
57112,'71_(film),Birdman_(film),12.0,other,12.0,1.000000
4283,'99–'00_Demos,Alive_'05,11.0,link,11.0,1.000000
11244,'It's_Alive!',Creature_of_Destruction,13.0,link,13.0,1.000000


### What is this code doing (#2) ?

In [70]:
node = "Alive"
n=10

In [75]:
prev

,prev,curr,n,type
53356,other-wikipedia,Alive,2221485,other
53343,other-empty,Alive,22109,other
53347,Main_Page,Alive,540,other
53345,Alive,Alive,215,other
53355,other-google,Alive,140,other
53354,LAN_Chile_Flight_210,Alive,98,other
53348,other-other,Alive,42,other
53357,other-yahoo,Alive,40,other
53349,other-bing,Alive,26,other
53353,Live,Alive,23,other


In [83]:
prev = df[df['curr'] == node].sort_values(by=['n'], ascending=False)[:n]
tuples = [list(x) for x in prev.values]
from pprint import pprint 
pprint(tuples)

[['other-wikipedia', 'Alive', 2221485, 'other'],
 ['other-empty', 'Alive', 22109, 'other'],
 ['Main_Page', 'Alive', 540, 'other'],
 ['Alive', 'Alive', 215, 'other'],
 ['other-google', 'Alive', 140, 'other'],
 ['LAN_Chile_Flight_210', 'Alive', 98, 'other'],
 ['other-other', 'Alive', 42, 'other'],
 ['other-yahoo', 'Alive', 40, 'other'],
 ['other-bing', 'Alive', 26, 'other'],
 ['Live', 'Alive', 23, 'other']]


In [84]:
prev = df[df['prev'] == node].sort_values(by=['n'], ascending=False)[:n]
tuples = [list(x) for x in prev.values]
from pprint import pprint 
pprint(tuples)

[['Alive', 'Alive_(1993_film)', 1948, 'link'],
 ['Alive', 'Alive:_The_Story_of_the_Andes_Survivors', 482, 'link'],
 ['Alive', 'Alive', 215, 'other'],
 ['Alive', 'Bat_Out_of_Hell_III:_The_Monster_Is_Loose', 202, 'link'],
 ['Alive', 'Alive:_20_Years_Later', 106, 'link'],
 ['Alive', 'Alive_(Pearl_Jam_song)', 97, 'link'],
 ['Alive', 'Alive!_(Kiss_album)', 69, 'link'],
 ['Alive', 'Life', 68, 'link'],
 ['Alive', 'Alive_(2002_film)', 51, 'link'],
 ['Alive', 'Alive_(Natalie_Bassingthwaighte_song)', 48, 'link']]


# Simple Network Analysis

We can think of Wikipedia as a network with articles as nodes and links between articles as edges. With the clickstream data we can assign weights to the edges, which correspond to how often the edges where traversed. Remember, that if an edge was traversed less than ten times in the month, it is not included in the data. The data set is on the large side, but with some patience, we can load it into networkx and look at some basic properties of the network. 
*takes five-fifteen seconds...*

In [85]:
import networkx as nx
clickstream = nx.DiGraph()

for i, row in df_post.iterrows():
    clickstream.add_edge(row['prev'], row['curr'], traffic = row['n'])

In [86]:
clickstream.number_of_nodes ()

70022

In [96]:
[n for n in clickstream.neighbors('10th_Academy_Awards')]

['11th_Academy_Awards',
 '1st_Academy_Awards',
 'A_Damsel_in_Distress_(film)',
 'A_Star_Is_Born_(1937_film)',
 'Academy_Award_for_Best_Picture',
 'Anne_Shirley_(actress)',
 'Barbara_Stanwyck',
 'Black_Legion_(film)',
 'Bob_Burns_(comedian)',
 'Captains_Courageous_(1937_film)',
 'Charles_Boyer',
 'Educated_Fish',
 'In_Old_Chicago',
 'Joseph_Schildkraut',
 'Luise_Rainer',
 'Paul_Muni',
 'Penny_Wisdom',
 'Ralph_Bellamy',
 'Snow_White_and_the_Seven_Dwarfs_(1937_film)',
 'Spencer_Tracy',
 'Stella_Dallas_(1937_film)',
 'The_Good_Earth_(film)',
 'The_Hurricane_(1937_film)',
 'The_Life_of_Emile_Zola',
 'The_Old_Mill',
 'The_Prisoner_of_Zenda_(1937_film)',
 'Torture_Money',
 'Waikiki_Wedding',
 'William_A._Wellman',
 'Wings_over_Honolulu',
 '12th_Academy_Awards',
 '87th_Academy_Awards',
 '9th_Academy_Awards',
 'Academy_Award_for_Best_Animated_Short_Film',
 'Alice_Brady',
 'Camille_(1936_film)',
 'Conquest_(1937_film)',
 'Dead_End_(1937_film)',
 'H._B._Warner',
 'Janet_Gaynor',
 'Los_Angeles_flo

In [97]:
[n for n in clickstream.neighbors('Spencer_Tracy')]

['40th_Academy_Awards', 'Academy_Awards']

In [93]:
[n for n in clickstream.neighbors('Bette_Davis')]

['Academy_Awards']

In [98]:
[n for n in clickstream.neighbors('Academy_Awards')]

['11th_Academy_Awards',
 '12_Years_a_Slave_(film)',
 '13th_Academy_Awards',
 '15th_Academy_Awards',
 '17th_Academy_Awards',
 '1997_Academy_Awards',
 '19th_Academy_Awards',
 '1st_Academy_Awards',
 '2002_Academy_Awards',
 '2008_Academy_Awards',
 '2011_Academy_Awards',
 '2013_Academy_Awards',
 '2014_in_film',
 '21st_Academy_Awards',
 '22nd_Academy_Awards',
 '24th_Academy_Awards',
 '26th_Academy_Awards',
 '28th_Academy_Awards',
 '2nd_Academy_Awards',
 '33rd_Academy_Awards',
 '35_mm_film',
 '35th_Academy_Awards',
 '37th_Academy_Awards',
 '39th_Academy_Awards',
 '40th_Academy_Awards',
 '41st_Academy_Awards',
 '42nd_Academy_Awards',
 '44th_Academy_Awards',
 '46th_Academy_Awards',
 '48th_Academy_Awards',
 '4th_Academy_Awards',
 '53rd_Academy_Awards',
 '55th_Academy_Awards',
 '57th_Academy_Awards',
 '59th_Academy_Awards',
 '60th_Academy_Awards',
 '61st_Academy_Awards',
 '62nd_Academy_Awards',
 '64th_Academy_Awards',
 '66th_Academy_Awards',
 '66th_Directors_Guild_of_America_Awards',
 '68th_Acade